# **1. Mô tả ứng dụng** 

*   **Ứng dụng mà nhóm chọn:** Huấn luyện Convolutional Neural Network để phân lớp ảnh bằng Stochastic Gradient Descent.
   *   **Input:**

   *   **Output:**

   *   **Ý nghĩa của ứng dụng trong thực tế:**

*   **Ứng dụng này có cần phải tăng tốc không:** có, vì ...

# **2. Cài đặt tuần tự**

*   **Thiết kế:**
*   **Đánh giá:**


# **3. Cài đặt song song (trên GPU)**

*   **Phân tích:**
*   **Thiết kế:**
*   **Đánh giá:**

# **4. Cài đặt song song (trên GPU) + tối ưu hóa**

# **5. Nhìn lại quá trình làm đồ án**

# **6. Tài liệu tham khảo**

https://www.kaggle.com/cdeotte/how-to-choose-cnn-architecture-mnist

https://towardsdatascience.com/wtf-is-image-classification-8e78a8235acb